<a href="https://colab.research.google.com/github/JanakievskaA/Vnp_project/blob/master/Standarden_proekt_solved.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install tensorflow tsfresh

In [ ]:
# Import all the libraries needed
import pandas as pd
from tsfresh.feature_extraction import extract_features
from tsfresh.feature_extraction.settings import MinimalFCParameters
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN

In [ ]:
# Преземање на податоците за акциите на Adidas (симбол: ADS.DE за германската берза)
# stock_data = yf.download('ADS.DE', start='2020-01-01', end='2024-01-01')
# stock_data.to_csv('adidas_stock_data.csv')

# Read the excel files
df_adidas_stock = pd.read_excel('/content/drive/MyDrive/adidas_stock_data.xlsx')
df_esg_score = pd.read_excel('/content/drive/MyDrive/esg_score.xlsx')

In [ ]:
df_adidas_stock

,Year,Open,High,Low,Close,Adj Close,Volume
0,2020-01-02,289.200012,293.250000,289.200012,291.549988,281.179169,429314
1,2020-01-03,290.000000,292.250000,288.350006,291.950012,281.564941,440067
2,2020-01-06,288.950012,290.649994,285.149994,289.649994,279.346710,494435
3,2020-01-07,290.549988,296.500000,290.500000,294.750000,284.265350,489629
4,2020-01-08,294.000000,302.850006,293.500000,302.850006,292.077240,686646
...,...,...,...,...,...,...,...
1016,2023-12-21,195.039993,197.100006,193.259995,194.419998,193.829056,386794
1017,2023-12-22,183.000000,184.679993,181.679993,184.139999,183.580292,1035464
1018,2023-12-27,185.419998,186.520004,184.820007,185.559998,184.995987,337533
1019,2023-12-28,185.679993,185.919998,182.860001,184.100006,183.540421,323489


In [ ]:
df_esg_score

,Date,ESG score
0,2020,79
1,2021,85
2,2022,89
3,2023,52


In [ ]:
df_adidas_stock.describe()

,Year,Open,High,Low,Close,Adj Close,Volume
count,1021,1021.000000,1021.000000,1021.000000,1021.000000,1021.000000,1.021000e+03
mean,2022-01-01 07:51:04.054848,219.570147,222.444505,216.657375,219.499226,214.323976,6.554253e+05
min,2020-01-02 00:00:00,95.989998,96.769997,93.400002,93.949997,93.272156,0.000000e+00
25%,2021-01-05 00:00:00,168.059998,170.940002,165.320007,168.839996,167.244568,4.151050e+05
50%,2022-01-04 00:00:00,215.899994,218.399994,211.600006,214.899994,208.425140,5.434480e+05
75%,2022-12-30 00:00:00,277.700012,280.299988,274.200012,277.600006,268.111145,7.571790e+05
max,2023-12-29 00:00:00,325.750000,336.250000,324.899994,336.250000,327.720154,4.104049e+06
std,NaN,60.144473,60.368940,59.777463,59.997458,56.569288,4.328440e+05


In [ ]:
df_adidas_stock.isnull().sum()

Year         0
Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64

In [ ]:
# Convert to CSV
df_adidas_stock.to_csv('/content/drive/MyDrive/adidas_stock_data.csv', index=False)
df_esg_score.to_csv('/content/drive/MyDrive/esg_score.csv', index=False)

In [ ]:
# Load data from CSV
df_adidas_stock = pd.read_csv('/content/drive/MyDrive/adidas_stock_data.csv')
df_esg_score = pd.read_csv('/content/drive/MyDrive/esg_score.csv')

In [ ]:
# Merge data on the year
df_adidas_stock['Year'] = pd.to_datetime(df_adidas_stock['Year']).dt.year
df_combined = pd.merge(df_adidas_stock, df_esg_score, how='left', left_on='Year', right_on='Date')

In [ ]:
# Check for missing values
df_combined.isnull().sum()

Year         0
Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
Date         0
ESG score    0
dtype: int64

In [ ]:
# Feature extraction using tsfresh with minimal settings
extracted_features = extract_features(df_combined.drop(columns=['ESG score']), column_id='Year', default_fc_parameters=MinimalFCParameters())

Feature Extraction: 100%|██████████| 28/28 [00:00<00:00, 1705.89it/s]


In [ ]:
# # Ensure that the DataFrame index is reset if necessary
# extracted_features.reset_index(drop=True, inplace=True)

In [ ]:
# Merge extracted features with ESG scores
df_features = extracted_features.merge(df_esg_score, left_index=True, right_on='Date')

In [ ]:
# Data preparation for modeling
X = df_features.drop(columns=['ESG score', 'Date'])
y = df_features['ESG score']

In [ ]:
# Normalize features
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

In [ ]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [ ]:
# Linear Regression Model
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)
y_pred_lr = lr_model.predict(X_test)
mse_lr = mean_squared_error(y_test, y_pred_lr)
print('Linear Regression MSE:', mse_lr)

Linear Regression MSE: 441.6867036114995


In [ ]:
# Random Forest Model
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)
y_pred_rf = rf_model.predict(X_test)
mse_rf = mean_squared_error(y_test, y_pred_rf)
print('Random Forest MSE:', mse_rf)

Random Forest MSE: 212.8681000000001


In [ ]:
# Gradient Boosting Model
gb_model = GradientBoostingRegressor(n_estimators=100, random_state=42)
gb_model.fit(X_train, y_train)
y_pred_gb = gb_model.predict(X_test)
mse_gb = mean_squared_error(y_test, y_pred_gb)
print('Gradient Boosting MSE:', mse_gb)

Gradient Boosting MSE: 320.0672046061515


In [ ]:
# Recurrent Neural Network Model
X_train_rnn = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
X_test_rnn = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

In [ ]:
rnn_model = Sequential([
    SimpleRNN(50, input_shape=(1, X_train.shape[1]), activation='relu'),
    Dense(1)
])
rnn_model.compile(optimizer='adam', loss='mean_squared_error')
rnn_model.fit(X_train_rnn, y_train, epochs=50, batch_size=10, verbose=1)
y_pred_rnn = rnn_model.predict(X_test_rnn)
mse_rnn = mean_squared_error(y_test, y_pred_rnn.flatten())
print('RNN MSE:', mse_rnn)

Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - loss: 5567.7407
Epoch 2/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step - loss: 5545.2485
Epoch 3/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - loss: 5522.6157
Epoch 4/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - loss: 5500.9102
Epoch 5/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - loss: 5479.6328
Epoch 6/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 136ms/step - loss: 5458.3521
Epoch 7/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - loss: 5436.5835
Epoch 8/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 153ms/step - loss: 5414.8477
Epoch 9/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 141ms/step - loss: 5392.8511
Epoch 10/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step - loss: 5370.7300
Epoch 11/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 131ms/step - loss: 5348.4351
Epoch 12/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 159ms/step - loss: 5325.6768
Epoch 13/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - loss: 5303.2959
Epoch 14/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 158ms/step - loss: 5280.9336
Epoch 15/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - loss: 5258.4

In [ ]:
# Function to evaluate the model
def evaluate_model(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    print(f"MSE: {mse}, RMSE: {rmse}, MAE: {mae}, R²: {r2}")

In [ ]:
# Evaluate each model
print("Linear Regression Evaluation:")
evaluate_model(y_test, y_pred_lr)

print("Random Forest Evaluation:")
evaluate_model(y_test, y_pred_rf)

print("Gradient Boosting Evaluation:")
evaluate_model(y_test, y_pred_gb)

print("RNN Evaluation:")
evaluate_model(y_test, y_pred_rnn.flatten())

Linear Regression Evaluation:
MSE: 441.6867036114995, RMSE: 21.016343726050437, MAE: 21.016343726050437, R²: nan
Random Forest Evaluation:
MSE: 212.8681000000001, RMSE: 14.590000000000003, MAE: 14.590000000000003, R²: nan
Gradient Boosting Evaluation:
MSE: 320.0672046061515, RMSE: 17.890422147231504, MAE: 17.890422147231504, R²: nan
RNN Evaluation:
MSE: 5287.2135590039, RMSE: 72.71322822570801, MAE: 72.71322822570801, R²: nan


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:996: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:996: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:996: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:996: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
